# Tensor-Flow

## Daniel Wilcox: 19147414

This example problem can be found within chapter 13 of the "Hands-on Machine Learning with Scikit-Learn and TensorFlow" by Aurélien Géron. 


In [1]:
import tensorflow as tf

import time
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_moons

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Exercises
## 7
### 7.1
Build your own CNN and try to achieve the highest possible accuracy on MNIST.

In [2]:
#Mnist Data:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_test = y_test.astype(np.int32)


In [3]:
#CNN

#Input information:
height = 28
width = 28
n_inputs = height*width

channel = 1 #grey scale

#Layer information:
#------------------

#Convolutional Layer 1:
conv1_fmaps = 32
conv1_ksize = 3#5
conv1_stride = 1
conv1_pad = "SAME"

#Convolutional Layer 2:
conv2_fmaps = 64#32
conv2_ksize = 3#5
conv2_stride = 1
conv2_pad = "SAME"

#Convolutional Layer 3:
conv3_fmaps = 64
conv3_ksize = 3
conv3_stride = 1
conv3_pad = "SAME"

#Convolutional Layer 4:
conv4_fmaps = 64
conv4_ksize = 3
conv4_stride = 1
conv4_pad = "SAME"

#Dropout rate:
drop_1 = 0.25 #dropout percentage = 25%

#Fully connected layer: 256, relu
n_neurons = 128#256
activation=tf.nn.relu
drop_2 = 0.5 #dropout percentage = 50%

#Output Layer:
n_outputs = 10
#------------------


In [4]:
reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channel])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')

    
with tf.name_scope("Conv_Layer_1"):
    
    #output: [1, 28, 28, 1] as stride is 1
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1") 

    #output: [1, 28, 28, 1] as stride is 1
    conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2") 
    

#with tf.name_scope("pool_1"):
#    #output: [1, 14, 14, 1] as stride is 2 (28/2 = 14)
#    pool_1 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

    
#---
with tf.name_scope("pool_2"):
    #output: [1, 14, 14, 1] as stride is 1
    pool_2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")    
    pool2_flat = tf.reshape(pool_2, shape=[-1, conv4_fmaps * 14 * 14])
    pool2_flat_drop = tf.layers.dropout(pool2_flat, drop_2, training=training)
#---
    
#with tf.name_scope("Conv_Layer_2"):
#    
#    #output: [1, 14, 14, 1] as stride is 1
#    conv3 = tf.layers.conv2d(pool_1, filters=conv3_fmaps, kernel_size=conv3_ksize,
#                         strides=conv3_stride, padding=conv3_pad,
#                         activation=tf.nn.relu, name="conv3")
#
#    #output: [1, 14, 14, 1] as stride is 1
#    conv4 = tf.layers.conv2d(conv3, filters=conv4_fmaps, kernel_size=conv4_ksize,
#                         strides=conv4_stride, padding=conv4_pad,
#                         activation=tf.nn.relu, name="conv4")
#    
#       
#with tf.name_scope("pool_2"):
#    #output: [1, 7, 7, 1] as stride is 1
#    pool_2 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")    
#    pool2_flat = tf.reshape(pool_2, shape=[-1, conv4_fmaps * 7 * 7])
#    pool2_flat_drop = tf.layers.dropout(pool2_flat, drop_1, training=training)

    
with tf.name_scope("fc_layer"):
    fc1 = tf.layers.dense(pool2_flat_drop, n_neurons, activation=activation, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, drop_2, training=training)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


In [ ]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)
    
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
n_epochs = 1000
batch_size = 50
iteration = 0

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            iteration += 1
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: X_valid, y: y_valid})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch {}\t last batch accuracy: {:.4f}%\t Valid acc: {:.4f}%\tValid best loss: {:.6f}".format(
                  epoch, acc_batch * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./my_mnist_model")

Epoch 0	 last batch accuracy: 100.0000%	 Valid acc: 98.2800%	Valid best loss: 0.066600
Epoch 1	 last batch accuracy: 96.0000%	 Valid acc: 98.8000%	Valid best loss: 0.047791
Epoch 2	 last batch accuracy: 98.0000%	 Valid acc: 97.3400%	Valid best loss: 0.038420
Epoch 3	 last batch accuracy: 100.0000%	 Valid acc: 98.9200%	Valid best loss: 0.038420
Epoch 4	 last batch accuracy: 100.0000%	 Valid acc: 99.2200%	Valid best loss: 0.032639
Epoch 5	 last batch accuracy: 100.0000%	 Valid acc: 99.0200%	Valid best loss: 0.032639
Epoch 6	 last batch accuracy: 98.0000%	 Valid acc: 99.2200%	Valid best loss: 0.031798
Epoch 7	 last batch accuracy: 100.0000%	 Valid acc: 99.3000%	Valid best loss: 0.031798
Epoch 8	 last batch accuracy: 100.0000%	 Valid acc: 99.2000%	Valid best loss: 0.030354
Epoch 9	 last batch accuracy: 100.0000%	 Valid acc: 98.9400%	Valid best loss: 0.030354
Epoch 10	 last batch accuracy: 100.0000%	 Valid acc: 99.3000%	Valid best loss: 0.030354
Epoch 11	 last batch accuracy: 100.0000%	 Val